In [ ]:
import seaborn as sns
import pandas as pd
import networkx as nx

from utilities import load_graph_from_files

graph, _ = load_graph_from_files('netherlands')

# run network stats on loaded graph
indegree = nx.in_degree_centrality(graph)
outdegree = nx.out_degree_centrality(graph)

# put indegree in a dataframe and map outdegree to it
df = pd.DataFrame(indegree.items(), columns=['id_str', 'indegree'])
df.set_index(['id_str'])
df['outdegree'] = df['id_str'].map(outdegree)

In [ ]:
# look up follower count and merge with degree stats

from utilities import db_to_pandas

def get_all_followers_count():
    sql_query="""
    SELECT 
    tweet_body->'user'->>'followers_count' as followers_count,
    tweet_body->'user'->>'id_str' as id_str
    FROM tweet
    WHERE location_id like 'netherlands%'
    AND tweet_body->>'lang' = 'nl'
    ORDER BY id_str, followers_count ASC;
    """
    q = db_to_pandas(sql_query)
    return q

df1 = df
df1 = df1.astype({'id_str': str})
df1.set_index(['id_str'])

df2 = get_all_followers_count().astype({'id_str': str, 'followers_count': int})
df2.groupby(['id_str'], sort=False)['followers_count'].max()
df2.set_index(['id_str'])

merged = pd.merge(df1, df2, on='id_str')
# keep last followers_count
merged = merged.drop_duplicates(subset=['id_str', 'indegree', 'outdegree'], keep='last')



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

sns.set(style="white")
plt.title("Interactions initiated vs interactions received with number of followers")
palette = sns.cubehelix_palette(start=2, rot=0, dark=0, light=.5, as_cmap=True)

ax = sns.scatterplot(data=merged, 
                     x="outdegree", y="indegree",
                     hue='followers_count',
                     size='followers_count',
                     sizes=(30, 1000), 
                     #alpha=0.1,
                     #legend='brief',
                     palette=palette
                  )
ax.set(xlim=(-0.01, None))
ax.set(ylim=(-0.0002, 0.005))
ax.set(xlabel='Out-degree (interactions received)', ylabel="In-degree (interactions initiated)")
sns.despine(offset=20, trim=True);

plt.savefig('outputs/outdegree_indegree_plot.png')